In [1]:
from pycaret.classification import *
import pandas as pd
import os
import wandb

os.chdir('/home/antoine/gene_pheno_pred')
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["WANDB_API_KEY"]="4e5748d6c6f3917c78cdc38a516a1bac776faf58"
wandb.login()
wandb.init(
    project="cigap-classif",
)


wandb: Currently logged in as: antoine-toffano (mlb). Use `wandb login --relogin` to force relogin


In [2]:
# Load training set
df1 = pd.read_csv('/home/antoine/gene_pheno_pred/TorusE_2023-03-31_10-18-16_kg_val.csv' , header=0)
df2 = pd.read_csv('/home/antoine/gene_pheno_pred/TorusE_2023-03-31_10-18-16_kg_test.csv' , header=0)
df = pd.concat([df1, df2], axis=0)

df['link'] = df['relation'].apply(lambda x: 1 if x != 'no_link_known' else 0)
df

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,head,relation,tail,link
0,-0.022025,0.016013,0.008182,-0.016668,-0.017219,-0.015423,0.014788,-0.013625,0.004813,-0.010436,...,0.012363,0.012363,-0.012363,0.013047,0.012363,-0.009570,https://wormbase.org/wbdata/ortho_dis-5315,no_link_known,https://wormbase.org/species/c_elegans/gene/WB...,0
1,0.011442,0.023409,0.027197,-0.026088,0.011466,0.011485,-0.011461,-0.027663,-0.021202,-0.024895,...,0.013770,0.008617,-0.013770,0.019268,0.013770,-0.022629,https://wormbase.org/wbdata/pheno-125368,no_link_known,https://wormbase.org/species/c_elegans/gene/WB...,0
2,-0.002173,-0.002173,-0.002173,0.002173,-0.002173,-0.002173,0.002173,0.002173,0.002173,0.003111,...,-0.000944,0.011679,0.000944,-0.006678,-0.001933,0.000131,https://wormbase.org/wbdata/pheno-256301,no_link_known,https://wormbase.org/wbdata/pheno-405138,0
3,0.017005,0.017386,0.017313,-0.017392,0.017594,0.017359,-0.017394,-0.017409,-0.017698,-0.016948,...,0.018851,0.016371,-0.018774,-0.017279,0.018851,-0.004702,https://wormbase.org/wbdata/ortho_dis-26094,http://www.semanticweb.org/needed-terms#001,https://wormbase.org/species/c_elegans/gene/WB...,1
4,-0.009484,0.003228,-0.006519,-0.001198,-0.009484,-0.000704,0.006519,0.000694,0.004623,0.000704,...,-0.009442,-0.016714,0.013269,-0.011895,-0.011088,0.005852,https://wormbase.org/wbdata/pheno-366163,no_link_known,https://wormbase.org/wbdata/pheno-196950,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234345,-0.003296,0.003037,-0.001115,0.007448,-0.001115,-0.007448,-0.003037,0.001115,0.007448,0.007448,...,-0.007389,0.013422,0.002157,-0.013422,-0.007389,-0.007389,https://wormbase.org/wbdata/pheno-76741,no_link_known,https://wormbase.org/wbdata/pheno-80813,0
234346,0.000480,0.000517,0.000911,0.003378,0.004769,-0.008777,-0.000911,-0.000480,0.008814,0.008777,...,0.013418,0.009380,-0.017695,0.021031,0.013418,-0.022710,https://wormbase.org/wbdata/pheno-397090,http://semanticscience.org/resource/SIO_000281,https://wormbase.org/species/all/phenotype/WBP...,1
234347,0.001610,0.005263,0.000994,0.000131,0.005880,-0.009614,-0.001610,-0.000994,0.014961,0.010230,...,0.019138,0.016125,-0.019138,0.015792,0.016125,-0.004549,https://wormbase.org/wbdata/pheno-37510,http://semanticscience.org/resource/SIO_001279,https://wormbase.org/species/all/phenotype/WBP...,1
234348,-0.029945,-0.008771,0.006061,-0.006316,-0.018599,0.029698,-0.007143,0.017606,-0.015266,-0.025612,...,-0.002534,0.008700,0.002534,-0.008700,-0.002534,-0.002534,https://wormbase.org/species/c_elegans/gene/WB...,no_link_known,https://wormbase.org/wbdata/pheno-95971,0


In [3]:
df2 = df

In [4]:
df = df.drop('relation', axis=1)

In [ ]:
# Experiment setup
s = setup(df, target = 'link', fold_strategy = 'stratifiedkfold', fold=10, train_size = 0.8, n_jobs=-1, system_log=True, use_gpu = True, log_experiment='wandb')
exp = ClassificationExperiment()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

# K Neighbors Classifier

In [ ]:
knn = create_model('knn')

In [ ]:
plot_model(knn, plot = 'confusion_matrix')

In [ ]:
plot_model(knn, plot = 'error')

In [ ]:
plot_model(knn, plot = 'parameter')

In [ ]:
save_model(knn, 'models_classif/convkb/binary_emb_val/knn')

# Naive Bayes

In [ ]:
nb = create_model('nb')

In [ ]:
plot_model(nb, plot = 'confusion_matrix')

In [ ]:
plot_model(nb, plot = 'error')

In [ ]:
plot_model(nb, plot = 'parameter')

In [ ]:
save_model(nb, 'models_classif/convkb/binary_emb_val/nb')

# Decision Tree Classifier

In [ ]:
dt = create_model('dt')

In [ ]:
plot_model(dt, plot = 'confusion_matrix')

In [ ]:
plot_model(dt, plot = 'error')

In [ ]:
plot_model(dt, plot = 'parameter')

In [ ]:
save_model(dt, 'models_classif/convkb/binary_emb_val/dt')

# SVM - Linear Kernel

In [ ]:
svm = create_model('svm')

In [ ]:
plot_model(svm, plot = 'confusion_matrix')

In [ ]:
plot_model(svm, plot = 'error')

In [ ]:
plot_model(svm, plot = 'parameter')

In [ ]:
save_model(svm, 'models_classif/convkb/binary_emb_val/svm')

# Ridge Classifier

In [ ]:
ridge = create_model('ridge')

In [ ]:
plot_model(ridge, plot = 'confusion_matrix')

In [ ]:
plot_model(ridge, plot = 'error')

In [ ]:
plot_model(ridge, plot = 'parameter')

In [ ]:
save_model(ridge, 'models_classif/convkb/binary_emb_val/ridge')

# Random Forest

In [ ]:
rf = create_model('rf')

In [ ]:
plot_model(rf, plot = 'confusion_matrix')

In [ ]:
plot_model(rf, plot = 'error')

In [ ]:
plot_model(rf, plot = 'parameter')

In [ ]:
save_model(rf, 'models_classif/convkb/binary_emb_val/rf')

# Quadratic Discriminant Analysis

In [ ]:
qda = create_model('qda')

In [ ]:
plot_model(qda, plot = 'confusion_matrix')

In [ ]:
plot_model(qda, plot = 'error')

In [ ]:
plot_model(qda, plot = 'parameter')

In [ ]:
save_model(qda, 'models_classif/convkb/binary_emb_val/qda')

# Linear Discriminant Analysis

In [ ]:
lda = create_model('lda')

In [ ]:
plot_model(lda, plot = 'confusion_matrix')

In [ ]:
plot_model(lda, plot = 'error')

In [ ]:
plot_model(lda, plot = 'parameter')

In [ ]:
save_model(lda, 'models_classif/convkb/binary_emb_val/lda')

# Extra Trees

In [ ]:
et = create_model('et')

In [ ]:
plot_model(et, plot = 'confusion_matrix')

In [ ]:
plot_model(et, plot = 'error')

In [ ]:
plot_model(et, plot = 'parameter')

In [ ]:
save_model(et, 'models_classif/convkb/binary_emb_val/et')

# Light Gradient Boosting Machine

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
plot_model(lightgbm, plot = 'confusion_matrix')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot = 'parameter')

In [ ]:
save_model(lightgbm, 'models_classif/convkb/binary_emb_val/lightgbm')

# MLP Classifier

In [ ]:
mlp = create_model('mlp')

In [ ]:
plot_model(mlp, plot = 'confusion_matrix')

In [ ]:
plot_model(mlp, plot = 'error')

In [ ]:
plot_model(mlp, plot = 'parameter')

In [ ]:
save_model(mlp, 'models_classif/convkb/binary_emb_val/mlp')